In [ ]:
# install the required librairies
!pip -q install transformers -U > /dev/null
!pip -q install bitsandbytes -U > /dev/null
!pip -q install accelerate -U > /dev/null

In [ ]:
#reboot Colab
import os
os.kill(os.getpid(), 9)

In [ ]:
#necessary packages
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
import torch
import accelerate

In [ ]:
# Load the model Quantized, this requires a GPU A100 , click the top right top corner, change runtime and select the GPU (could get those A100 with Colab + paid suscription, or by using Google Cloud)
model_name_or_path = "mistralai/Mixtral-8x7B-Instruct-v0.1"
config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
config.max_position_embeddings = 8096
quantization_config = BitsAndBytesConfig(
bnb_4bit_compute_dtype=torch.bfloat16,
load_in_4bit=True,
bnb_4bit_quant_type='nf4',
bnb_4bit_use_double_quant=True,
llm_int8_enable_fp32_cpu_offload=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

In [ ]:
# Load the model tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
model_name_or_path,
config=config,
trust_remote_code=True,
quantization_config=quantization_config,
device_map="auto",
offload_folder="./offload"
)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Let's prompt the model
prompt = "[INST] What are the 3 main groups that are representative of the french touch electro music scene? [/INST] "
print(f"""###prompt:\n{prompt}\n### Mixtral 8x7B Instruct:""")
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to("cuda")
streamer = TextStreamer(tokenizer, skip_prompt= True)
output = model.generate(
    **inputs,
    streamer=streamer,
    max_new_tokens=256,
    temperature=0.8,
    do_sample=True,
    top_k=20,
    top_p=0.4,
    repetition_penalty=1.2
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


###prompt:
[INST] What are the 3 main groups that are representative of the french touch electro music scene? [/INST] 
### Mixtral 8x7B Instruct:
1. Daft Punk: This French electronic duo is one of the most well-known and successful acts in the French Touch movement. They have been credited with helping to bring the genre to mainstream audiences around the world, thanks to their distinctive sound which blends elements of house, disco, and funk.
2. Justice: Another highly influential group from the French Touch scene, Justice consists of two producers who create a high-energy style of dance music characterized by heavy basslines, distorted synths, and rock 'n' roll influences. Their debut album "†" (Cross) was released in 2007 and went on to become an international hit.
3. Cassius: Formed in the mid-90s, Cassius is considered one of the pioneering acts of the French Touch movement. The duo, composed of Philippe Zdar and Boom Bass, helped define the early sound of the genre with their ble

In [ ]:
text = "<s> [INST] What are the 3 main groups that are representative of the french touch electro music scene? [/INST]"
inputs = tokenizer(text, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=256)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))